### PromptTemplate 
* [PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)
* [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
* [ChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatMessagePromptTemplate.html#langchain_core.prompts.chat.ChatMessagePromptTemplate)
* [FewShotPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate)
* PartialPrompt

In [1]:
# poetry add python-dotenv langchain langchain-openai

In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_T


##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    #model="openai/gpt-oss-120b",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)

/Users/eon/skRookies/myLangChain/mylangchain-app/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


('ChatGPT는 인터넷의 방대한 텍스트에서 단어와 문장의 등장 패턴을 학습합니다.  \n'
 '학습 과정에서 입력된 문맥에 따라 다음에 올 단어를 예측하도록 훈련됩니다.  \n'
 '이러한 확률 기반 예측 방식으로, 자연스럽고 유창한 대화를 생성할 수 있게 됩니다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [4]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="한국어")
print(combined_prompt)

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"한국어"})

pprint(response)

input_variables=['count', 'language', 'model_name'] input_types={} partial_variables={} template='{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요.\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요.'
('1. ChatGPT는 인터넷 글을 대량으로 읽고 다음에 올 단어를 맞히는 ‘예측’ 훈련을 되풀이해 스스로 언어 규칙을 익힙니다.  \n'
 '2. 사람이 질문-답을 평가해 좋은 대답에 점수를 주면, 이 점수로 모델을 다시 조정해 정답 방향을 학습합니다.  \n'
 '3. 위 두 과정을 거듭해 모델은 맥락을 파악하고 자연스러운 대화를 만들어내게 됩니다.\n'
 '\n'
 'ChatGPT의 장점 요약  \n'
 '- 다양한 주제에 대해 논리적이고 일관된 글을 즉시 생성  \n'
 '- 질의의 맥락을 기억하며 이어지는 대화 가능  \n'
 '- 코드 작성·번역·요약 등 다재다능하며, 영어는 물론 한국어 등 여러 언어에 능숙  \n'
 '- 추가 학습·미세 조정으로 특정 업무(고객 응대, 자료 요약 등)에 특화 가능  \n'
 '- 대화 형태라 초보자도 별도 코딩 없이 바로 활용 가능  \n'
 '\n'
 'ChatGPT와 비슷한 AI 모델  \n'
 '구글 바드, 클로드, LLaMA, PaLM, 코그뷰, 블룸')


#### PromptTemplate 의 파라미터를 배열 형태로 하여 여러개 사용하는 경우

In [5]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 3},
    {"model_name": "Gemini", "count": 4},
    {"model_name": "Claude", "count": 4}
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions] #파이썬에서 파라미터로 딕셔너리 받을 때 아스타(*) 2개 씀
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    print(type(prompt), prompt)
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.', 'Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.', 'Claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.']
<class 'str'> GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.
('GPT-4는 방대한 텍스트 데이터에서 다음 단어를 예측하도록 Transformer 신경망을 학습합니다.  \n'
 '학습 과정에서 단어의 위치와 문맥을 함께 고려해 자기회귀 방식으로 가중치를 조정합니다.  \n'
 '이렇게 얻은 확률 분포를 바탕으로 인간이 이해할 수 있는 자연스러운 문장을 생성합니다.')
<class 'str'> Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Gemini는 텍스트·이미지·오디오·코드 등 다양한 데이터를 동시에 처리하는 멀티모달 아키텍처로, Transformer 기반의 '
 '인코더-디코더 구조를 확장해 학습합니다.  \n'
 '대규모 코퍼스에서 다음 토큰 예측과 동시에 교차 모달 정렬 작업을 수행해 서로 다른 데이터 간 의미를 정렬합니다.  \n'
 '학습 과정에서 전문가 혼합(MoE) 기법으로 일부 네트워크만 활성화해 계산 효율을 높이고, 강화학습과 인간 피드백(RLHF)으로 응답 '
 '품질을 추가로 향상시킵니다.  \n'
 '결과적으로 Gemini는 단일 모델로 텍스트 생성, 이미지 이해, 코드 작성 등 다양한 작업을 통합적으로 수행할 수 있습니다.')
<class 'str'> Claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Claude는 방대한 텍스트 데이터를 학습해 언어의 확률 분포를 익히는 트랜스포머 기반 대규모 언어 모델입니다.  \n'
 '학습 단계에서는 수십억 개의 파라미터를 조정해 다음 토큰이 무엇일지 예측하는 방식으로 최적화됩니다.  \n'
 '인간 피드백 강화학습(RLHF) 단계에서는 사람이 선호하는 답변 순위를 바탕으로 보상 모델을 만들

##### 2) ChatPromptTemplate
* Tuple 형태의 system, user, assistant 메시지 지원
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 간결성과 가독성이 높고 단순한 구조

In [6]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
print(messages)

# 생성한 메시지를 바로 주입하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

print(type(response))
print(response.content)

[SystemMessage(content='This system is an expert in answering questions about AI. Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
ChatGPT는 “GPT(Generative Pre-trained Transformer)” 계열의 언어 모델로,  
“거대한 텍스트를 보고 다음 단어를 예측하도록 학습한 뒤, 사람이 좋아할 행동을 최대화하도록 미세 조정했다”는 한 줄 요약을 풀어 쓴 것이 전체 학습 원리입니다. 아래에 단계별로, 수식·코드 없이 핵심 메커니즘을 설명합니다.

-------------------------------------------------
1. 두 단계 파이프라인
1) Pre-training(사전 학습)  
2) RLHF 미세 조정(후속 학습)  
두 단계가 모두 끝나야 ‘채팅처럼 말을 잇는’ ChatGPT가 됩니다.

-------------------------------------------------
2. 사전 학습(Pre-training) 단계
목표  
“주어진 앞 문장을 보고, 다음에 나올 단어의 확률 분포를 맞춰라.”  
즉, ‘자연어를 압축해서 내부 파라미터에 저장하라’는 뜻입니다.

데이터  
웹, 위키, 책 등 1조 토큰(단어·부분 단어) 이상.

모델 구조  
트랜스포머 디코더(Transformer decoder) 블록 96개, 1750억 개 가중치.  
Self-attention이 문맥 안의 모든 토큰 간 관계를 한 번에 계산해 ‘글로벌 맥락’을 잡습니다.

학습 방법  
1) 랜덤 초

In [7]:
# 체인을 생성하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

<class 'str'>
ChatGPT는 “GPT” 계열 모델의 대화 특화판이므로, GPT가 어떻게 학습하는지를 먼저 이해해야 합니다. GPT-3.5·GPT-4 같은 모델은 3단계 파이프라인으로 만들어집니다.

1단계: Pre-training (사전학습)  
2단계: Supervised Fine-tuning (SFT, 지도 미세조정)  
3단계: Reinforcement Learning with Human Feedback (RLHF, 인간 피드백 기반 강화학습)

아래에 각 단계에서 어떤 데이터·목적함수·알고리즘이 쓰이는지, 그리고 왜 “대화”가 가능해지는지를 수식과 함께 설명합니다.

-------------------------------------------------
1. Pre-training: “인터넷 글을 읽으며 다음 토큰 예측하기”
-------------------------------------------------
목표  
최대한 많은 텍스트에서 통계적 패턴을 포착해 “주어진 앞 토큰 시퀀스 x₁…xₜ가 있을 때, 다음 토큰 xₜ₊₁의 확률”을 정확히 추정하는 것입니다.

모델 구조  
- Transformer 디코더(마스크드 셀프어텐션)  
- 입력 임베딩 + 위치 임베딩  
- L개의 트랜스포머 블록 → 최종 은닉 상태 h_L  
- 언어 모델 헤드: Pθ(xₜ₊₁|x≤ₜ) = softmax(W·h_L)

학습 목적함수  
L(θ) = – Σ_{t=1}^{T} log Pθ(xₜ | x₁…xₜ₋₁)  
즉, 전체 시퀀스에 대한 음의 로그 우도를 최소화합니다.

데이터  
웹페이지, 책, 위키피디아, GitHub 코드 등 수천억 토큰.  
GPT-3.5는 300B 토큰, GPT-4는 아직 공개되지 않았지만 그 이상으로 추정됩니다.

결과  
이 시점에서 모델은 “글쓰기 스타일”은 잘 흉내 내지만, 질문-답 형식은 거의 없고, 안전성/유용성도 낮습니다.

-------------------------------------------------

#### 3) ChatPromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [8]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

#### ChatMessagePromptTemplate는 여러 종류의 메시지(시스템, 인간, AI)를 조합하여 복잡한 프롬프트를 생성할 때 유용합니다.
* SystemMessagePromptTemplate: 이 템플릿은 AI 모델에게 역할을 부여하거나 전반적인 규칙을 설정하는 시스템 메시지를 만듭니다. 위의 예시에서는 "번역을 도와주는 유용한 도우미"라는 역할을 지정합니다.
* HumanMessagePromptTemplate: 이 템플릿은 사용자의 질문이나 요청을 담는 인간 메시지를 만듭니다. 아래의 예시에서는 번역할 텍스트를 입력받습니다.
* ChatPromptTemplate.from_messages: 이 클래스 메서드는 시스템 메시지, 인간 메시지 등 여러 종류의 MessagePromptTemplate 객체들을 리스트로 받아 하나의 채팅 프롬프트 템플릿으로 통합합니다.
* format_messages: 이 메서드는 정의된 템플릿에 실제 값을 채워 넣어 [SystemMessage, HumanMessage] 형태의 리스트를 반환합니다. 이 리스트는 채팅 모델(Chat Model) 에 바로 전달될 수 있습니다.

In [9]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. SystemMessagePromptTemplate와 HumanMessagePromptTemplate 생성
# SystemMessagePromptTemplate는 모델의 페르소나 또는 기본 지침을 설정합니다.
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# HumanMessagePromptTemplate는 사용자로부터 받는 입력 프롬프트를 정의합니다.
human_template = "{text_to_translate}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 2. ChatPromptTemplate 생성
# 위에서 만든 두 템플릿을 리스트로 묶어 ChatPromptTemplate을 만듭니다.
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. 프롬프트 포맷팅
# chat_prompt_template.format_messages()를 사용하여 최종 메시지 리스트를 생성합니다.
# 이 함수는 딕셔너리 형태의 입력 변수를 받습니다.
formatted_prompt = chat_prompt_template.format_messages(
    input_language="English",
    output_language="Korean",
    text_to_translate="I love programming."
)

# 4. 결과 출력
print(formatted_prompt)

# LLM 호출
response = llm.invoke(formatted_prompt)

# 결과 출력
print(response.content)

[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]
나는 프로그래밍을 사랑해요.


##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

##### 4-1) PromptTemplate을 사용하지 않는 경우

In [10]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
#llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
태양계 행성(태양에서부터의 순서)

1. 수성(Mercury) – 가장 작고, 가까우며, 표면 온도 변화가 극심  
2. 금성(Venus) – 두꺼운 이산화탄소 대기, 지름·질량은 지구와 비슷하지만 표면 460 °C  
3. 지구(Earth) – 유일한 액체 해양과 생명체 보유  
4. 화성(Mars) – 붉은 사막·극冠, 얇은 대기, 탐사 로봇 상주  
5. 목성(Jupiter) – 태양계 최대 기체 행성, 대적점·79개 이상 위성(가니메데 등)  
6. 토성(Saturn) – 눈에 띄는 고리계, 밀도는 물보다 낮음, 80개 이상 위성(타이탄 등)  
7. 천왕성(Uranus) – 옆으로 누운 자전축, 얼음·가스 혼합, 약한 고리  
8. 해왕성(Neptune) – 태양계 궤도가 가장 긴 기체 행성, 강한 바람(초속 600 m)

※ 작은 행성(지구형) 4개 → 큰 행성(목성형) 4개  
※ 2006년 명왕성은 ‘왜행성’으로 분류됨


##### 4-2) FewShotChatMessagePromptTemplate 사용하는 경우

In [11]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(chain)

# 테스트 실행
result = chain.invoke({"input": "양자컴퓨팅에 대하여 설명해 주세요."})
print(result.content)

first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '뉴턴의 운동 법칙을 요약해 주세요.', 'output': '### 뉴턴의 운동 법칙\n1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.\n2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.\n3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다.'}, {'input': '지구의 대기 구성 요소를 알려주세요.', 'output': '### 지구 대기의 구성\n- **질소 (78%)**: 대기의 대부분을 차지합니다.\n- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.\n- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.\n- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다.'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[

#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [12]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = llm.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

 프롬프트: 가을에 일어나는 대표적인 지구과학 현상은 태풍 발생이 맞나요? 가을에 주로 발생하는 지구과학 현상을 3개 알려주세요
 모델 응답: 가을에 **"태풍 발생"**은 **정확히 맞지 않습니다**.  
태풍은 **여름철(7~9월)**에 **가장 많이 발생**하지만, **가을에도 발생할 수는 있습니다**—특히 9~10월에는 **고도 발달한 강력한 태풍(예: 가을 태풍)**이 자주 오지만, **가을에 "주로" 발생하는 현상은 아닙니다**.

---

### ✅ 가을에 **주로 발생하는 대표적인 지구과학 현상 3가지**:

1. **대기 안정화로 인한 **가을 고기압** 형성**  
   → 여름철의 불안정한 대기가 안정되면서 **고기압이 강화**되고, **맑고 건조한 날씨**가 계속됨.

2. **기온 역전 현상 (Radiation inversion)**  
   → 밤이 길어지고 지면이 빠르게 식으면서 **지표 근처의 기온이 상층보다 낮아지는 현상**이 빈번해짐.  
   → **안개**, **서리**, **이슬**이 자주 발생.

3. **단풍과 낙엽의 계절적 변화 (식생 계절성)**  
   → 기온 하강과 일조 시간 감소로 **나무의 활동이 줄어들고**, **잎의 색소 변화(단풍)**가 일어남.  
   → 이는 **기후학적 계절 변화**의 일부로, **지구과학적으로도 중요한 현상**입니다.

---

### 요약
- 태풍은 **여름철 주요 현상**, 가을엔 **드물게 강력한 가을 태풍**이 올 뿐.
- 가을에는 **고기압 강화**, **기온 역전**, **단풍/낙엽** 같은 **계절적 지구과학 현상**이 대표적입니다.


In [13]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# Step 1: 현재 계절 결정
season_name = get_current_season()  # 계절 값 얻기
print(f"현재 계절: {season_name}")

현재 계절: 가을


In [14]:

# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 사용
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
##llm = ChatOpenAI(
##    #api_key=OPENAI_API_KEY,
##    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
##    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
##    temperature=0.0
##)

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season": lambda x : season_name}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season_name}에 발생하는 자연 현상:\n{response}")


 가을에 발생하는 자연 현상:
가을철에 뚜렷하게 나타나는 지구과학 현상 3가지를 꼽으면 다음과 같습니다.

1. 시베리아 고기압의 발달  
   아시아 대륙이 급속히 식으면서 시베리아-몽골 부근에 강한 고기압이 형성됩니다. 이 고기압이 남하하면서 한반도에 맑고 건조한 가을 날씨를 만들고, 남쪽으로 진출할 때는 겨울철 북서 계절풍의 ‘전초부’ 역할을 합니다.

2. 기압골 주도의 이상고온(가을 늦더위)  
   중국 대륙에서 동진하는 기압골이 한반도 상공을 지나면 대기가 강한 하강气流(하강气流)를 타고 압축·가열됩니다. 이때 하층의 동풍이 뜨거진 야산성(야간 산성) 기류를 내륙으로 끌어들여 9∼10월에 30 ℃ 안팎의 ‘가을 늦더위’를 발생시킵니다.

3. 산간 계곡에서의 복사(輻射) 안개  
   밤이 길어지고 지면이 맑게 감기면서 계곡 바닥은 대기보다 훨씬 차가워집니다. 이에 따라 인근 공기가 수증화 포화 상태에 이르러 수증기가 응결·부유하면서 연무(안개)가 깊게 깔리는 현상이 자주 관측됩니다.


#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [15]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 프롬프트 전달 및 응답 받기
response = llm.invoke(prompt.format())

# 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)

 프롬프트: 현재 1달러 = 1406.48원 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.
 모델 응답: 현재 환율 1달러 = 1,406.48원은 **역대 최고 수준**에 근접한 수치로, 단순한 '약세'를 넘어 **심각한 원화 취약 증후군**이 본격화된 신호로 읽히는 구간입니다. 아래에 5가지 측면에서 구조적·정책적 함의를 정리해 드립니다.

---

### 1. 수치로 본 위치
- **고점 대비 상승률**: 2017년 1,050원 대비 약 34% 절상(원화 약세)  
- **작년 동기 대비**: 2023년 5월 1,320원 → 1년 새 약 6.5% 추가 약세  
- **트리플 디지트 심리선**: 1,400원 돌파는 서울·워싱턴 양쪽 모두 '사안'이 될 수준

---

### 2. 원화 약세의 3대 구조적 책임 요인
| 요인 | 설명 | 비고 |
|---|---|---|
| **1) 수출 구조 전환** | 반도체·배터리 등 대형 품목이 수출 둔화, 중국 베이스 이격↑ | 한국은행 기준, 2024년 1-4월 수출 YoY -6.5% |
| **2) 안전자산 선호** | 미국 10년물 4.5% vs 한국 10년물 3.5% → 차익거래 자금 지속 유출 | 외국인 채권투자 2024년 누적 –8조원 |
| **3) 경상수지 구조 변화** | 2022년 이후 에너지·식품 역외 가격↑에도 불구, 재정건전성 우선 → 재정소비 억제 | 2023년 경상수지 흑자 29.8bn$ → 2024년 13bn$ 전망(하락) |

---

### 3. 금통위·재정당국의 선택지
1. **외환건전성 부담**  
   - 한국은행이 1,400원 '심리선'을 사실상 방치 → 시장 "라인 인 더 샌드" 신뢰 하락  
   - 실효성 있는 방어는 1) 1조+ 달러 매도(외환보유액 4,100억$ 대비 2.5% 수준) 2) 양적 긴축(통화안정증권 발행 확대) 3) 재정·금융당국 합동 '변동성 완화' 선언  

2. **물가·가계부담**  
   - 원화 약세 10% → 수입물가지수 